In [1]:
# Generate Spark Session with Iceberg configuration
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, lit, struct

spark = SparkSession.builder \
    .appName("SocketStreamingToIceberg") \
    .master("local[*]") \
    .config("spark.ui.port", "4041") \
    .config("spark.sql.streaming.checkpointLocation", "checkpoint") \
    .config("spark.sql.adaptive.enabled", "false") \
    .getOrCreate()
spark

24/05/29 20:30:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Read streaming data from sockets (maintain error handling)
try:
    df_raw = spark.readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", "9999") \
        .load()
except Exception as e:
    print(f"Error reading from socket: {e}")
    exit(1)

24/05/29 19:44:55 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
print("Streaming data received from socket!")

df_raw.printSchema()
distData = sc.parallelize(data)


Streaming data received from socket!
root
 |-- value: string (nullable = true)



In [4]:
# Split the line into words and handle potential exceptions
from pyspark.sql.functions import split

try:
    df_words = df_raw.withColumn("words", split("value", " "))
except Exception as e:
    print(f"Error splitting words: {e}")
    exit(1)

In [5]:
# Explode the list of words (maintain error handling)
from pyspark.sql.functions import explode

try:
    df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")
except Exception as e:
    print(f"Error exploding words: {e}")
    exit(1)

In [6]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")


In [7]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [8]:
# Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+----+---+
|word|cnt|
+----+---+
+----+---+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+---+
| word|cnt|
+-----+---+
|hello|  2|
|world|  1|
+-----+---+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 